In [ ]:
!git clone https://github.com/mymusise/ChatGLM-Tuning.git
%cd  ChatGLM-Tuning
!pip install -r requirements.txt 

In [1]:
from modeling_chatglm import ChatGLMForConditionalGeneration
import torch


torch.set_default_tensor_type(torch.cuda.HalfTensor)
model = ChatGLMForConditionalGeneration.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True, device_map='auto')

/home/mymusise/pro/stable-diffusion-webui/venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
Loading checkpoint shards: 100%|██████████| 8/8 [00:05<00:00,  1.46it/s]


In [2]:
from peft import PeftModel

model = PeftModel.from_pretrained(model, "mymusise/chatGLM-6B-alpaca-lora")
torch.set_default_tensor_type(torch.cuda.FloatTensor)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: /usr/local/cuda-11.5/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 115
CUDA SETUP: Loading binary /home/mymusise/pro/stable-diffusion-webui/venv/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda115.so...


In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True)

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


In [5]:
from cover_alpaca2jsonl import format_example

# alpaca数据集
instructions = [
 {'instruction': 'Give three tips for staying healthy.',
  'input': '',
  'output': '1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.',
 },
 {'instruction': 'What are the three primary colors?',
  'input': '',
  'output': 'The three primary colors are red, blue, and yellow.',
 }]

# alpaca数据集，问题翻译为中文，output为chatgpt的输出
instructions += [
 {'instruction': '请给出三个健康生活的建议',
  'input': '',
  'output': '1.保持均衡的饮食：饮食是影响身体健康的一个重要因素。\n2.定期运动：运动对身体健康有很多好处。\n3.睡眠充足：良好的睡眠质量对身体健康至关重要。',
 },
 {'instruction': '三原色是什么？(请用中文)',
  'input': '',
  'output': '三原色是指色彩三基色，即红色、绿色和蓝色。在色彩学中，通过不同比例的三原色的叠加可以形成各种色彩，这也是彩色显示技术和印刷技术的基础',
 },
 {'instruction': '以下哪家公司类型与其它的不同',
  'input': '微博、京东、淘宝',
  'output': '微博',
 }
]
with torch.no_grad():
    for idx, item in enumerate(instructions):
        feature = format_example(item)
        input_text = feature['context']
        ids = tokenizer.encode(input_text)
        input_ids = torch.LongTensor([ids])
        out = model.generate(
            input_ids=input_ids,
            max_length=150,
            do_sample=False,
            temperature=0
        )
        out_text = tokenizer.decode(out[0])
        answer = out_text.replace(input_text, "").replace("\nEND", "").strip()
        item['infer_answer'] = answer
        print(out_text)
        print(f"### {idx+1}.Answer:\n", item.get('output'), '\n\n')

Instruction: Give three tips for staying healthy.
Answer: 1. Eat a balanced diet of fruits, vegetables, lean protein, and whole grains.
2. Get regular exercise, such as walking, running, or swimming.
3. Stay hydrated by drinking plenty of water.
### 1.Answer:
 1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. 
2. Exercise regularly to keep your body active and strong. 
3. Get enough sleep and maintain a consistent sleep schedule. 


Instruction: What are the three primary colors?
Answer: The three primary colors are red, blue, and yellow.
### 2.Answer:
 The three primary colors are red, blue, and yellow. 


Instruction: 请给出三个健康生活的建议
Answer: 1. 坚持运动,每周至少进行150分钟的中等强度有氧运动。 
2. 饮食健康,多吃蔬菜、水果、全谷物、蛋白质和健康脂肪。 
3. 睡眠充足,每天保证7-9小时的睡眠时间。
### 3.Answer:
 1.保持均衡的饮食：饮食是影响身体健康的一个重要因素。
2.定期运动：运动对身体健康有很多好处。
3.睡眠充足：良好的睡眠质量对身体健康至关重要。 


Instruction: 三原色是什么?(请用中文)
Answer: 三原色是指红、黄、蓝三种颜色。
### 4.Answer:
 三原色是指色彩三基色，即红色、绿色和蓝色。在色彩学中，通过不同比例的三原色的叠加可以形成各种色彩，这也是彩色显示技术和印刷技术的基础 


Instruc